# LLMs are pure functions

In [1]:
from pydantic import BaseModel, Field
import json

from basic.api_key import ensure_api_key
ensure_api_key()
from openai import OpenAI
client = OpenAI()

OpenAI API key has been configured.


In [2]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="What is the current price of Bitcoin in USD?",
)
print(response.output_text)

[2025-09-14 17:56:46 - httpx:1025 - INFO] HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


I can't provide real-time data, including current cryptocurrency prices. You can check the latest price of Bitcoin on financial news websites, cryptocurrency exchanges, or financial apps.


# Calling functions

## Mock function

In [3]:
from basic.utils import get_current_price
get_current_price("EUR/USD")

Fetching current price for EUR/USD...


{'pair': 'EUR/USD',
 'base': 'EUR',
 'quote': 'USD',
 'price': 3434.33,
 'timestamp': '2025-09-14T17:56:46.834664'}

In [4]:
class ToolCall(BaseModel):
    tool_name: str = Field(description="The name of the tool to be called. The available tools are: get_current_price.")
    tool_input: str = Field(description="The input to be passed to the tool. For the get_current_price tool, this should be the currency pair, e.g., BTC/USD.")

class Reply(BaseModel):
    answer: str | None = Field(description="The answer to the question in a concise manner.")
    tool_call: ToolCall | None = Field(description="If the answer requires calling a tool, this field will contain the tool call details.")

In [5]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that is good with finance."},
    {"role": "user", "content": "What is the current price of Bitcoin in USD?"},
]
response = client.chat.completions.parse(
    model="gpt-4o-mini",
    messages=messages,
    response_format=Reply
)
response_text = response.choices[0].message.content
reply = Reply.model_validate_json(response_text)
messages.append({"role": "assistant", "content": response_text})
reply

[2025-09-14 17:56:49 - httpx:1025 - INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Reply(answer=None, tool_call=ToolCall(tool_name='get_current_price', tool_input='BTC/USD'))

In [6]:
if reply.tool_call is not None:
    if reply.tool_call.tool_name == "get_current_price":
        current_price = get_current_price(reply.tool_call.tool_input)
        messages.append({"role": "user", "content": json.dumps(current_price)})
        response = client.chat.completions.parse(
            model="gpt-4o-mini",
            messages=messages,
            response_format=Reply
        )
        response_text = response.choices[0].message.content
        reply = Reply.model_validate_json(response_text)
        messages.append({"role": "assistant", "content": response_text})

        print(reply)
    else:
        print("Some other tool has been called.")
else:
    print("No luck this time. Try again.")

Fetching current price for BTC/USD...


[2025-09-14 17:56:51 - httpx:1025 - INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


answer='The current price of Bitcoin (BTC) in USD is $37,243.92.' tool_call=None


## OpenAI function calling

In [22]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_price",
            "description": "Get the current price of a cryptocurrency trading pair",
            "parameters": {
                "type": "object",
                "properties": {
                    "pair": {
                        "type": "string",
                        "description": "The trading pair in format BASE/QUOTE, e.g. BTC/USD, ETH/USD"
                    }
                },
                "required": ["pair"],
                "additionalProperties": False
            }
        }
    }
]

messages = [
    {"role": "system", "content": "You are a helpful assistant that is good with finance."},
    {"role": "user", "content": "What is the current price of Bitcoin in USD?"},
]

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages,
    tools=tools
)
response_message = response.choices[0].message
messages.append(response_message)

if response_message.tool_calls:
    tool_call = response_message.tool_calls[0]
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    if function_name == "get_current_price":
        function_response = get_current_price(**function_args)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(function_response)
        })
        resp = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages
        )
        resp_message = resp.choices[0].message
        print(resp_message.content)
        messages.append({"role": "assistant", "content": resp_message.content})

[2025-09-14 18:28:23 - httpx:1025 - INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Fetching current price for BTC/USD...


[2025-09-14 18:28:24 - httpx:1025 - INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The current price of Bitcoin (BTC) in USD is approximately $19,478.29.
